## Broward County Midterm Election Results — Governor and Lieutenant Governor (2022)

By: Shirsho Dasgupta (2022)

The code in this repository scrapes precinct-level election results for the Governor race in Broward county. 

The column names in the dataframe are generated only after the results have been scraped, meaning that the code is reusable for different races and the dataframe dynamically sizes itself according to the number of candidates or issues on the ballot. It does not matter how many candidates there are or in which order they appear. 

The code then computes the vote shares of each candidate, generates the names of the candidates who are leading (or have won) and are runners-up, their party affiliations and their lead/win margin in each precinct. It also computes the leader/winner and their vote-share and lead/win margin for the entire county. 

The resulting dataframe for each race is then joined with previously prepared spreadsheets containing precinct-level demographic and voter registration data.

A sample final dataframe can be found [here](https://github.com/shirshod/florida_midterms_2022/blob/main/precinct_results/broward/governor_broward/scraper_files/gov_broward_scraper_report_11-14-2022-2100.csv).

A data dictionary can be found [here](https://github.com/shirshod/florida_midterms_2022/blob/main/precinct_results/broward/broward_precinct_dictionary.pdf). 

The final dataframe was the foundation of and/or used in this story and table:
1. [Florida Latinos catapulted Republicans in the 2022 election. Are they the outliers?](https://www.miamiherald.com/news/politics-government/article268644252.html)
2. [Vote shares in Hispanic-majority precincts of select Florida counties](https://www.datawrapper.de/_/72L8M/)

#### Sources:
[Broward County Elections Department — Election Results](https://enr.electionsfl.org/BRO/3281/Precincts/47440/0/354/)

[Broward County Elections Department — Precinct Demographics and Voter Registrations](https://www.browardvotes.gov/Portals/Broward/Documents/2022Elections/October-Active%20Voters%20by%20District-Precinct.pdf)

##### Note:
Websites showing live results generally switch over to the next election and may not be displaying the data for the elections being discussed here. Please use [Wayback Machine](https://archive.org/web/) to view archived versions of the dataframe.

### Importing libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import datetime as dt
import os

In [2]:
### ignores copy warning
pd.options.mode.chained_assignment = None 

### Scraping results at sub-precinct level

In [3]:
### creates directory to store final dataframe after exporting
os.makedirs("governor_broward/scraper_files/", exist_ok = True)

### sets URL
url = "https://enr.electionsfl.org/BRO/3281/Precincts/47440/0/354/"

### retrieves data
page = requests.get(url)
### converts data to text for reading
soup = BeautifulSoup(page.text)

### stores all div-tags with class name "Race row"
divs_race = soup.find_all("div", {"class": "Race row"})

### creates list to write data into later
all_rows = []

### loop runs through all the stored divs
for i in range(0, len(divs_race)):
    
    ### stores precinct name
    base_precinct_no = divs_race[i].find("span", {"class": "PrecinctName"}).text
    
    ### stores all the div-tags with the candidate names
    names_tag = divs_race[i].find_all("td", {"class": "ChoiceColumn"}) 
    ### stores number of candidates
    candidate_len = len(names_tag)
    
    ### stores all the div-tags with the votes
    results_tag = divs_race[i].find_all("td", {"class": "DetailResultsColumn notranslate TotalVotes"})
    
    ### loop runs through all the divs with the candidate names
    for j in range(0, len(names_tag)):
        
        ### creates list to write data into (in format: [precinct details, name, votes])
        row = []
        
        ### writes precinct details into list
        row.append(base_precinct_no)
        
        ### retrieves, stores and writes candidate name into list
        name = names_tag[j].text
        name = " ".join(name.split())
        row.append(name)
        
        ### retrieves, stores and writes votes obtained
        votes = results_tag[j].text.replace(',',"")
        row.append(votes)
        
        ### stores each list with precinct, candidate and vote details as elements of a master list — a list of lists
        all_rows.append(row)
    
    ### converts the master list into dataframe
    df_votes = pd.DataFrame(all_rows, columns = ["BASE_PRECINCT_NUMBER", "CANDIDATE", "VOTES"])

### displays dataframe
df_votes

,BASE_PRECINCT_NUMBER,CANDIDATE,VOTES
0,A001,Ron DeSantis (REP),612
1,A001,Charlie Crist (DEM),358
2,A001,Hector Roos (LPF),1
3,A001,Carmen Jackie Gimenez (NPA),2
4,A002,Ron DeSantis (REP),1783
...,...,...,...
1415,Z004,Carmen Jackie Gimenez (NPA),0
1416,Z073,Ron DeSantis (REP),43
1417,Z073,Charlie Crist (DEM),65
1418,Z073,Hector Roos (LPF),0


### Compiling results into precinct-level

In [4]:
### converts number of votes into integers
df_votes["VOTES"] = df_votes["VOTES"].astype(int)
### groups sub-precincts into base precincts
df_voteprecinct = df_votes.groupby(["BASE_PRECINCT_NUMBER", "CANDIDATE"]).agg({"VOTES": "sum"}).reset_index()

### loop runs through newly created dataframe
for i in range(0, len(df_voteprecinct)):
    ### converts base precinct numbers into strings
    df_voteprecinct["BASE_PRECINCT_NUMBER"][i] = str(df_voteprecinct["BASE_PRECINCT_NUMBER"][i])

### displays dataframe
df_voteprecinct

,BASE_PRECINCT_NUMBER,CANDIDATE,VOTES
0,A001,Carmen Jackie Gimenez (NPA),2
1,A001,Charlie Crist (DEM),358
2,A001,Hector Roos (LPF),1
3,A001,Ron DeSantis (REP),612
4,A002,Carmen Jackie Gimenez (NPA),13
...,...,...,...
1415,Z004,Ron DeSantis (REP),157
1416,Z073,Carmen Jackie Gimenez (NPA),0
1417,Z073,Charlie Crist (DEM),65
1418,Z073,Hector Roos (LPF),0


### Converting dataset into horizontal format

In [5]:
### initializes a counter to store number of rows already processed
counter = 0

### initializes new list (referred to as "final list" later) to write data into
final_rows = []

### loop runs through dataframe
for i in range(counter, len(df_voteprecinct)):
    
    ### stores base precinct number
    base_precinct = df_voteprecinct["BASE_PRECINCT_NUMBER"][i]
    
    ### stores time of processing/update
    tnow = dt.datetime.now()
    dt_string = tnow.strftime("%m-%d-%Y %H:%M")
    update = "Last updated at " + str(dt_string)
    
    ### creates new list with first element as processing time
    pre_row = [update]
    ### adds base precinct number to list
    pre_row.append(base_precinct)
    
    ### nested loop runs through dataframe but starts from number denoted by counter
    ### this ensures that rows already read and processed are not read again
    for j in range(counter, len(df_voteprecinct)):
        
        ### if the base precinct number matches, then candidate details are stored in list
        if df_voteprecinct["BASE_PRECINCT_NUMBER"][j] == base_precinct:
            
            ### adds candidate name to list
            c_name = df_voteprecinct["CANDIDATE"][j]
            c_name = c_name.replace("WRITE-IN", "Write-In")
            pre_row.append(c_name)
            ### adds votes won to list
            pre_row.append(df_voteprecinct["VOTES"][j])
            ### extracts party name and adds to list
            c_party = df_voteprecinct["CANDIDATE"][j][-4:][:-1]
            c_party = c_party.replace("E-I", "OTHER")
            pre_row.append(c_party)
            
            
            ### counter moves by one
            counter = counter + 1
            
            ### try-except block handles exception that will occur in last row of dataframe with below commands
            try:
                
                ### if counter is the same as length of dataframe, then last row is reached
                ### the list-element is added to the final list
                if counter == len(df_voteprecinct):
                    ### adds list to final list
                    final_rows.append(pre_row)
                    
                ### if base precinct number in next row is not same as base precinct number in current row,
                ### then details of that precinct has completely been recorded
                ### the list-element is added to the final list
                elif df_voteprecinct["BASE_PRECINCT_NUMBER"][j+1] != df_voteprecinct["BASE_PRECINCT_NUMBER"][j]:
                    ### adds list to final list
                    final_rows.append(pre_row)
                
                ### continues to next iteration
                else:
                    continue
            ### handles exception
            except:
                pass
    
    ### converts final list to dataframe
    df = pd.DataFrame(final_rows)

### displays dataframe
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Last updated at 11-14-2022 21:00,A001,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),358,DEM,Hector Roos (LPF),1,LPF,Ron DeSantis (REP),612,REP
1,Last updated at 11-14-2022 21:00,A002,Carmen Jackie Gimenez (NPA),13,NPA,Charlie Crist (DEM),910,DEM,Hector Roos (LPF),2,LPF,Ron DeSantis (REP),1783,REP
2,Last updated at 11-14-2022 21:00,A003,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),888,DEM,Hector Roos (LPF),9,LPF,Ron DeSantis (REP),1158,REP
3,Last updated at 11-14-2022 21:00,A004,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),780,DEM,Hector Roos (LPF),6,LPF,Ron DeSantis (REP),421,REP
4,Last updated at 11-14-2022 21:00,A005,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),1035,DEM,Hector Roos (LPF),8,LPF,Ron DeSantis (REP),190,REP


### Computing total votes tallied

In [6]:
### storing number of columns
col_no = len(df.columns)

### creates new column to write totals into at the end
### since this is an index-number, it automatically computes the required column number irrespective of number of candidates
df[col_no] = " "

### creates list to store tallied votes
votes_tallied = []

### loop runs through newly created horizontal dataframe
for i in range(0, len(df)):
    
    ### initializes variable to store total
    total = 0
    
    ### nested loop starts at column 3 (that is the first column with a vote number)
    ### loop iterates (number of candidates*3 + 1) at intervals of three to read all tallied votes
    ### note: each candidate has three corresponding elements—name,votes,party—hence the (n*3+1)
    for j in range(3, (candidate_len*3 + 1), 3):
        ### adds votes to total
        total = total + df[j][i]
    
    ### adds total to list
    votes_tallied.append(total)
    ### writes total into new column
    df[col_no][i] = total

### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Last updated at 11-14-2022 21:00,A001,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),358,DEM,Hector Roos (LPF),1,LPF,Ron DeSantis (REP),612,REP,973
1,Last updated at 11-14-2022 21:00,A002,Carmen Jackie Gimenez (NPA),13,NPA,Charlie Crist (DEM),910,DEM,Hector Roos (LPF),2,LPF,Ron DeSantis (REP),1783,REP,2708
2,Last updated at 11-14-2022 21:00,A003,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),888,DEM,Hector Roos (LPF),9,LPF,Ron DeSantis (REP),1158,REP,2063
3,Last updated at 11-14-2022 21:00,A004,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),780,DEM,Hector Roos (LPF),6,LPF,Ron DeSantis (REP),421,REP,1211
4,Last updated at 11-14-2022 21:00,A005,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),1035,DEM,Hector Roos (LPF),8,LPF,Ron DeSantis (REP),190,REP,1241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Last updated at 11-14-2022 21:00,Z001,Carmen Jackie Gimenez (NPA),18,NPA,Charlie Crist (DEM),1339,DEM,Hector Roos (LPF),7,LPF,Ron DeSantis (REP),122,REP,1486
351,Last updated at 11-14-2022 21:00,Z002,Carmen Jackie Gimenez (NPA),9,NPA,Charlie Crist (DEM),458,DEM,Hector Roos (LPF),2,LPF,Ron DeSantis (REP),558,REP,1027
352,Last updated at 11-14-2022 21:00,Z003,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),79,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),28,REP,107
353,Last updated at 11-14-2022 21:00,Z004,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),50,DEM,Hector Roos (LPF),0,LPF,Ron DeSantis (REP),157,REP,207


### Computing precinct-level vote shares, leads and margins

In [7]:
### creates new list to store index numbers of newly created columns
new_cols = []

### loop runs through number of candidates and four more for other details
for i in range(0, (candidate_len + 4)):   
    ### creates (n+4) new columns for "n" candidates
    df[col_no + i + 1] = " "
    ### stores numbers in a list as strings
    new_cols.append(str(col_no + i + 1))

### loop runs through each row of dataframe
for i in range(0, len(df)):
    
    ### creates lists to store details
    votes_obtained = []
    party = []
    can_names = []
    
    ### stores the total votes tallied for each row
    total = votes_tallied[i]
    
    ### checks whether votes have been tallied
    if total != 0: 
    
        ## TALLYING VOTES

        ### nested loop starts at column 3 (that is the first column with a vote number)
        ### loop iterates (number of candidates*3 + 1) at intervals of three to read all details
        ### note: each candidate has three corresponding elements—name,votes,party—hence the (n*3+1)
        for j in range(3, (candidate_len*3 + 1), 3):

            ### stores votes, party and candidate names in the relevant lists
            ### note: here format is [cols x rows] while traversing through dataframe
            votes_obtained.append(df[j][i])
            party.append(df[j+1][i])
            can_names.append(df[j-1][i])


        ## COMPUTING VOTE-SHARES

        ### loop runs through all the candidates
        for k in range(0, candidate_len):
            ### column-index of a candidate is given by corresponding element in "new_cols"
            ### first candidate (k = 0) will be stored in column at index-0 in the "new_cols" list and so on
            column_no = int(new_cols[k])
            ### computes percent and writes into the column
            df[column_no][i] = (votes_obtained[k]/total*100).round(2)

        ## COMPUTING LEADING CANDIDATES AND WIN MARGINS

        ### computes and stores the highest vote-tally for the "votes_obtained" list
        max_value = max(votes_obtained)

        ### checks if there are more elements with same value (in case of ties)
        if votes_obtained.count(max_value) > 1:

            ### method returns indices of all duplicate highest vote-tallies
            indices = []
            def find_indices(list_to_check, item_to_find):
                return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

            ### stores indices of all duplicate highest vote-tallies
            indices = find_indices(votes_obtained, max_value)

            ### constructs string of all candidate names corresponding to highest vote-tallies
            string = []
            for ind in range(0, len(indices)):
                index = indices[ind]
                string.append(can_names[index])

            ### stores value as "TIED"
            lead_party = "TIED"
            ### stores candidate names with tied votes
            lead_can = " and ".join([", ".join(string[:-1]),string[-1]] if len(string) > 2 else string)
            ### stores margin of difference as 0
            margin = 0
            margin_pp = 0

        else:
            ### computes and stores the index at which the highest vote is stored in the "votes_obtained" list
            max_index = votes_obtained.index(max_value)
            ### stores name of party corresponding to highest vote-tally in the fourth-last column
            ### if highest vote-tally is second (index=1) element in "votes_obtained"
            ### the second element (index=1) in "party" list is the corresponding datapoint
            lead_party = party[max_index]
            ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
            lead_can = can_names[max_index]

            ### computes and stores the second highest vote-tally for the "votes_obtained" list
            second_max_value = max(votes_obtained, key = lambda x: min(votes_obtained)-1 if (x == max_value) else x)
            ### computes and stores the raw number of votes between first and second placed candidates
            margin = max_value - second_max_value
            ### computes the difference in percent points between first and second placed candidates
            margin_pp = ((max_value/total*100)-(second_max_value/total*100)).round(2)

        ### stores index-number of the fourth-last element of "new_cols" list (this will be the fourth-last column)
        lead_col = int(new_cols[-4])
        ### stores name of party corresponding to highest vote-tally in the fourth-last column
        ### if highest vote-tally is second (index=1) element in "votes_obtained"
        ### the second element (index=1) in "party" list is the corresponding datapoint
        df[lead_col][i] = lead_party
        ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
        df[lead_col+1][i] = lead_can

        ### stores index-number of the penultimate element of "new_cols" list (this will be the penultimate column)
        raw_margin_col = int(new_cols[-2])
        ### stores raw margin of difference in penultimate column
        df[raw_margin_col][i] = margin

        ### stores index-number of the last element of "new_cols" list (this will be the last column)
        margin_pp_col = int(new_cols[-1])
        ### stores percent points difference in last column
        df[margin_pp_col][i] = margin_pp
    
    ### if no votes have been tallied
    else: 
        
        ### loop runs through all candidates
        for n in range(0, candidate_len):
            ### column-index of a candidate is given by corresponding element in "new_cols"
            ### first candidate (k = 0) will be stored in column at index-0 in the "new_cols" list and so on
            column_no = int(new_cols[n])
            ### stores vote-share as 0 and writes into the column
            df[column_no][i] = 0
        
        ### sets values 
        lead_party = "No votes tallied"
        lead_can = "No votes tallied"
        margin = 0
        margin_pp = 0
        
        ### stores index-number of the fourth-last element of "new_cols" list (this will be the fourth-last column)
        lead_col = int(new_cols[-4])
        ### stores name of party corresponding to highest vote-tally in the fourth-last column
        ### if highest vote-tally is second (index=1) element in "votes_obtained"
        ### the second element (index=1) in "party" list is the corresponding datapoint
        df[lead_col][i] = lead_party
        ### similarly, stores name of candidate corresponding to highest vote-tally in the next (third-last) column
        df[lead_col+1][i] = lead_can

        ### stores index-number of the penultimate element of "new_cols" list (this will be the penultimate column)
        raw_margin_col = int(new_cols[-2])
        ### stores raw margin of difference in penultimate column
        df[raw_margin_col][i] = margin

        ### stores index-number of the last element of "new_cols" list (this will be the last column)
        margin_pp_col = int(new_cols[-1])
        ### stores percent points difference in last column
        df[margin_pp_col][i] = margin_pp       

### replaces null values with "0"
df.fillna(0, inplace = True)    
   
### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,Last updated at 11-14-2022 21:00,A001,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),358,DEM,Hector Roos (LPF),1,...,REP,973,0.21,36.79,0.10,62.90,REP,Ron DeSantis (REP),254,26.10
1,Last updated at 11-14-2022 21:00,A002,Carmen Jackie Gimenez (NPA),13,NPA,Charlie Crist (DEM),910,DEM,Hector Roos (LPF),2,...,REP,2708,0.48,33.60,0.07,65.84,REP,Ron DeSantis (REP),873,32.24
2,Last updated at 11-14-2022 21:00,A003,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),888,DEM,Hector Roos (LPF),9,...,REP,2063,0.39,43.04,0.44,56.13,REP,Ron DeSantis (REP),270,13.09
3,Last updated at 11-14-2022 21:00,A004,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),780,DEM,Hector Roos (LPF),6,...,REP,1211,0.33,64.41,0.50,34.76,DEM,Charlie Crist (DEM),359,29.64
4,Last updated at 11-14-2022 21:00,A005,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),1035,DEM,Hector Roos (LPF),8,...,REP,1241,0.64,83.40,0.64,15.31,DEM,Charlie Crist (DEM),845,68.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Last updated at 11-14-2022 21:00,Z001,Carmen Jackie Gimenez (NPA),18,NPA,Charlie Crist (DEM),1339,DEM,Hector Roos (LPF),7,...,REP,1486,1.21,90.11,0.47,8.21,DEM,Charlie Crist (DEM),1217,81.90
351,Last updated at 11-14-2022 21:00,Z002,Carmen Jackie Gimenez (NPA),9,NPA,Charlie Crist (DEM),458,DEM,Hector Roos (LPF),2,...,REP,1027,0.88,44.60,0.19,54.33,REP,Ron DeSantis (REP),100,9.74
352,Last updated at 11-14-2022 21:00,Z003,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),79,DEM,Hector Roos (LPF),0,...,REP,107,0.00,73.83,0.00,26.17,DEM,Charlie Crist (DEM),51,47.66
353,Last updated at 11-14-2022 21:00,Z004,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),50,DEM,Hector Roos (LPF),0,...,REP,207,0.00,24.15,0.00,75.85,REP,Ron DeSantis (REP),107,51.69


### Computing overall county vote shares, leads and margins

In [8]:
## COMPUTING COUNTY-LEVEL TOTALS

### storing number of columns
new_col_no = len(df.columns)

total_col = candidate_len*3 + 2
df[new_col_no] = df[total_col].sum() 
ct_total = df[total_col].sum()

### creates new list to store index numbers of newly created columns
ct_cols = []

### stores new length of dataframe
ct_new_col_no = len(df.columns)

if ct_total != 0:

    ## COMPUTING COUNTY-LEVEL VOTE SHARES

    ### computes sum of all votes per candidate
    ct_can_votes = []
    for j in range(3, (candidate_len*3 + 1), 3):
        ct_can_votes.append(df[j].sum())

    ### computes vote-shares of candidates
    for i in range(0, candidate_len):
        df[ct_new_col_no + i] = (ct_can_votes[i]/ct_total*100).round(2)

    ## COMPUTING LEADING CANDIDATES AND WIN MARGINS

    ### computes new length of dataframe
    final_col_no = len(df.columns)

    ### stores overall maximum votes
    ct_max_value = max(ct_can_votes)

    ### if there are more than one instances of highest votes (in case of a tie)
    if ct_can_votes.count(ct_max_value) > 1:

        ### method returns indices of all duplicate highest vote-tallies
        indices = []
        def find_indices(list_to_check, item_to_find):
            return [idx for idx, value in enumerate(list_to_check) if value == item_to_find]

        ### stores indices of all duplicate highest vote-tallies
        indices = find_indices(ct_can_votes, ct_max_value)

        ### constructs string of all candidate names corresponding to highest votes tallied
        string = []
        for ind in range(0, len(indices)):
            index = indices[ind]
            string.append(can_names[index])

        ### stores leading party as tied
        ct_lead_party = "TIED"
        ### stores names of tied candidates
        ct_lead_can = " and ".join([", ".join(string[:-1]),string[-1]] if len(string) > 2 else string)

        ### stores margins as "0"
        ct_margin = 0
        ct_margin_pp = 0

        ### stores vote share for candidate with highest vote-tally
        ct_lead_voteshare = (ct_max_value/ct_total*100).round(2)

    else: 

        ### computes index of the highest tallied vote
        ct_max_index = ct_can_votes.index(ct_max_value)

        ### stores name of corresponding party and candidates
        ct_lead_party = party[ct_max_index]
        ct_lead_can = can_names[ct_max_index]

        ### computes second highest tallied vote
        second_ct_max_value = max(ct_can_votes, key = lambda x: min(ct_can_votes)-1 if (x == ct_max_value) else x)
        ### computes index of the second-highest tallied vote
        second_ct_max_index = ct_can_votes.index(second_ct_max_value)
        
        ### stores name of corresponding party and candidates
        second_ct_party = party[second_ct_max_index]
        second_ct_can = can_names[second_ct_max_index]

        ### stores margin of difference and the margin as percent points
        ct_margin = ct_max_value - second_ct_max_value
        ct_margin_pp = ((ct_max_value/ct_total*100)-(second_ct_max_value/ct_total*100)).round(2)

        ### stores the vote share of the highest tallied vote
        ct_lead_voteshare = (ct_max_value/ct_total*100).round(2)
        ### stores the vote share of the second highest tallied vote
        second_ct_voteshare = (second_ct_max_value/ct_total*100).round(2)

    ### writes into cells
    df[final_col_no + 1] = ct_lead_party
    df[final_col_no + 2] = ct_lead_can
    df[final_col_no + 3] = ct_margin
    df[final_col_no + 4] = ct_margin_pp
    df[final_col_no + 5] = ct_lead_voteshare
    df[final_col_no + 6] = second_ct_party
    df[final_col_no + 7] = second_ct_can
    df[final_col_no + 8] = second_ct_voteshare
    
else: 
    
    ### sets vote-shares of candidates as 0
    for i in range(0, candidate_len):
        df[ct_new_col_no + i] = 0
        
    ### computes new length of dataframe
    final_col_no = len(df.columns)
    
    ct_lead_party = "No votes tallied"
    ct_lead_can = "No votes tallied"
    ct_margin = 0
    ct_margin_pp = 0
    ct_lead_voteshare = 0
    second_ct_party = "No votes tallied"
    second_ct_can = "No votes tallied"
    second_ct_voteshare = 0
    
    ### writes into cells
    df[final_col_no + 1] = ct_lead_party
    df[final_col_no + 2] = ct_lead_can
    df[final_col_no + 3] = ct_margin
    df[final_col_no + 4] = ct_margin_pp
    df[final_col_no + 5] = ct_lead_voteshare
    df[final_col_no + 6] = second_ct_party
    df[final_col_no + 7] = second_ct_can
    df[final_col_no + 8] = second_ct_voteshare

### replaces null values with "0"
df.fillna(0, inplace = True)    
   
### displays dataframe
df

,0,1,2,3,4,5,6,7,8,9,...,26,27,29,30,31,32,33,34,35,36
0,Last updated at 11-14-2022 21:00,A001,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),358,DEM,Hector Roos (LPF),1,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
1,Last updated at 11-14-2022 21:00,A002,Carmen Jackie Gimenez (NPA),13,NPA,Charlie Crist (DEM),910,DEM,Hector Roos (LPF),2,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
2,Last updated at 11-14-2022 21:00,A003,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),888,DEM,Hector Roos (LPF),9,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
3,Last updated at 11-14-2022 21:00,A004,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),780,DEM,Hector Roos (LPF),6,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
4,Last updated at 11-14-2022 21:00,A005,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),1035,DEM,Hector Roos (LPF),8,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Last updated at 11-14-2022 21:00,Z001,Carmen Jackie Gimenez (NPA),18,NPA,Charlie Crist (DEM),1339,DEM,Hector Roos (LPF),7,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
351,Last updated at 11-14-2022 21:00,Z002,Carmen Jackie Gimenez (NPA),9,NPA,Charlie Crist (DEM),458,DEM,Hector Roos (LPF),2,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
352,Last updated at 11-14-2022 21:00,Z003,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),79,DEM,Hector Roos (LPF),0,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
353,Last updated at 11-14-2022 21:00,Z004,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),50,DEM,Hector Roos (LPF),0,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97


### Computing and inserting column names

In [9]:
### creates list to store column names with two initial elements
cols = ["LAST_UPDATE_2022", "BASE_PRECINCT"]

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column names for each candidate and adds to list
    cols.append("CANDIDATE_NAME_" + str(i+1))
    cols.append("VOTES_WON_" + str(i+1))
    cols.append("CANDIDATE_PARTY_" + str(i+1))

### adds column to list
cols.append("TOTAL_VOTES_TALLIED")

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column name for each candidate and adds to list
    cols.append("VOTE_SHARE_" + str(i+1))

### adds columns to list
cols.append("LEAD_PARTY")
cols.append("LEAD_CANDIDATE")
cols.append("RAW_MARGIN")
cols.append("MARGIN_PERCENTPOINTS")
cols.append("CT_TOTAL_VOTES_TALLIED")

### loop runs through all the candidates
for i in range(0, candidate_len):
    ### generates column names for each candidate and adds to list
    cols.append("CT_VOTE_SHARE_" + str(i+1))

cols.append("CT_LEAD_PARTY")
cols.append("CT_LEAD_CANDIDATE")
cols.append("CT_RAW_MARGIN")
cols.append("CT_MARGIN_PERCENTPOINTS")
cols.append("CT_LEAD_VOTESHARE")
cols.append("CT_SECOND_PARTY")
cols.append("CT_SECOND_CANDIDATE")
cols.append("CT_SECOND_VOTESHARE")

### replaces column indices with generated column names
df.columns = cols

### displays dataframe
df

,LAST_UPDATE_2022,BASE_PRECINCT,CANDIDATE_NAME_1,VOTES_WON_1,CANDIDATE_PARTY_1,CANDIDATE_NAME_2,VOTES_WON_2,CANDIDATE_PARTY_2,CANDIDATE_NAME_3,VOTES_WON_3,...,CT_VOTE_SHARE_3,CT_VOTE_SHARE_4,CT_LEAD_PARTY,CT_LEAD_CANDIDATE,CT_RAW_MARGIN,CT_MARGIN_PERCENTPOINTS,CT_LEAD_VOTESHARE,CT_SECOND_PARTY,CT_SECOND_CANDIDATE,CT_SECOND_VOTESHARE
0,Last updated at 11-14-2022 21:00,A001,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),358,DEM,Hector Roos (LPF),1,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
1,Last updated at 11-14-2022 21:00,A002,Carmen Jackie Gimenez (NPA),13,NPA,Charlie Crist (DEM),910,DEM,Hector Roos (LPF),2,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
2,Last updated at 11-14-2022 21:00,A003,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),888,DEM,Hector Roos (LPF),9,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
3,Last updated at 11-14-2022 21:00,A004,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),780,DEM,Hector Roos (LPF),6,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
4,Last updated at 11-14-2022 21:00,A005,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),1035,DEM,Hector Roos (LPF),8,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Last updated at 11-14-2022 21:00,Z001,Carmen Jackie Gimenez (NPA),18,NPA,Charlie Crist (DEM),1339,DEM,Hector Roos (LPF),7,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
351,Last updated at 11-14-2022 21:00,Z002,Carmen Jackie Gimenez (NPA),9,NPA,Charlie Crist (DEM),458,DEM,Hector Roos (LPF),2,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
352,Last updated at 11-14-2022 21:00,Z003,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),79,DEM,Hector Roos (LPF),0,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97
353,Last updated at 11-14-2022 21:00,Z004,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),50,DEM,Hector Roos (LPF),0,...,0.22,41.97,DEM,Charlie Crist (DEM),91992,15.37,57.34,REP,Ron DeSantis (REP),41.97


### Merging final dataframe with dataset of precinct demographics and prior results

In [10]:
### importing dataset with demographics and prior results
df_demos = pd.read_csv("broward_gov_demos.csv",  keep_default_na = False)

### merging 
df_master = pd.merge(df, df_demos, on = "BASE_PRECINCT")

### loop runs through dataframe
for i in range(0, len(df_master)):
    ### converts the precinct numbers to string
    df_master["BASE_PRECINCT"][i] = str(df_master["BASE_PRECINCT"][i])
    
### stores time
dt_string = tnow.strftime("%m-%d-%Y-%H%M")
### constructs filename
report = "gov_broward_scraper_report_" + str(dt_string) + ".csv"
path = "governor_broward/scraper_files/" + report

### exports dataframe 
df_master.to_csv(path, index = False)

### displays dataframe
df_master

,LAST_UPDATE_2022,BASE_PRECINCT,CANDIDATE_NAME_1,VOTES_WON_1,CANDIDATE_PARTY_1,CANDIDATE_NAME_2,VOTES_WON_2,CANDIDATE_PARTY_2,CANDIDATE_NAME_3,VOTES_WON_3,...,CT_SECOND_VOTESHARE,TOTAL_REGISTERED_VOTERS,RACE,PARTISANSHIP,DEMOCRAT_SHARE,REPUBLICAN_SHARE,NPA_SHARE,WHITE_SHARE,BLACK_SHARE,HISPANIC_SHARE
0,Last updated at 11-14-2022 21:00,A001,Carmen Jackie Gimenez (NPA),2,NPA,Charlie Crist (DEM),358,DEM,Hector Roos (LPF),1,...,41.97,1713,Majority White,Other,27.3,36.8,33.4,85.7,1.5,5.7
1,Last updated at 11-14-2022 21:00,A002,Carmen Jackie Gimenez (NPA),13,NPA,Charlie Crist (DEM),910,DEM,Hector Roos (LPF),2,...,41.97,4359,Majority White,Other,25.9,41.3,30.2,84.8,1.4,6.6
2,Last updated at 11-14-2022 21:00,A003,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),888,DEM,Hector Roos (LPF),9,...,41.97,4486,Majority White,Other,36.2,27.7,33.9,65.3,8.8,15.9
3,Last updated at 11-14-2022 21:00,A004,Carmen Jackie Gimenez (NPA),4,NPA,Charlie Crist (DEM),780,DEM,Hector Roos (LPF),6,...,41.97,3267,Other,Heavily Democrat,56.3,13.8,27.9,29.8,42.2,19.3
4,Last updated at 11-14-2022 21:00,A005,Carmen Jackie Gimenez (NPA),8,NPA,Charlie Crist (DEM),1035,DEM,Hector Roos (LPF),8,...,41.97,3563,Majority Black,Heavily Democrat,69.6,7.0,22.3,12.8,68.0,11.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Last updated at 11-14-2022 21:00,Z001,Carmen Jackie Gimenez (NPA),18,NPA,Charlie Crist (DEM),1339,DEM,Hector Roos (LPF),7,...,41.97,4684,Majority Black,Heavily Democrat,79.0,3.3,16.8,2.5,87.3,4.2
351,Last updated at 11-14-2022 21:00,Z002,Carmen Jackie Gimenez (NPA),9,NPA,Charlie Crist (DEM),458,DEM,Hector Roos (LPF),2,...,41.97,2865,Other,Other,42.8,22.1,33.7,28.1,16.3,42.9
352,Last updated at 11-14-2022 21:00,Z003,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),79,DEM,Hector Roos (LPF),0,...,41.97,396,Other,Other,43.7,13.4,40.7,21.0,6.6,6.6
353,Last updated at 11-14-2022 21:00,Z004,Carmen Jackie Gimenez (NPA),0,NPA,Charlie Crist (DEM),50,DEM,Hector Roos (LPF),0,...,41.97,409,Majority White,Other,23.5,44.5,29.8,76.5,4.2,8.6
